
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## HyCReeW runup estimation 

inputs required: 
  * Nearshore reconstructed historical storms
  * Nearshore reconstructed simulated storms
  * Historical water levels
  * Synthetic water levels 

in this notebook:
  * HyCReWW runup estimation of historical and synthetic events
  * Extreme value analysis and validation


### Workflow:

<div>
<img src="resources/nb02_04.png" width="400px">
</div>



**HyCReWW** provides wave-driven run-up estimations along coral reef-lined shorelines under a wide range of fringing reef morphologies and offshore forcing characteristics. The metamodel is based on two models: (a) a full factorial design of recent XBeach Non-Hydrostatic simulations under different reef configurations and offshore wave and water level conditions (Pearson et al, 2017); and (b) Radial Basis Functions (RBFs) for approximating the non-linear function of run-up for the set of multivariate parameters:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Runup = RBF($\eta_0$, $H_0$, ${H_0/L_0}$, $\beta_f$,$W_reef$, $\beta_b$, $c_f$ );&nbsp;&nbsp;&nbsp;&nbsp;</center>

Where, the hydrodynamic variables defined are offshore water level ($\eta_0$), significant wave height ($H_0$), and wave steepness (${H_0/L_0}$); the reef morphologic parameters include fore reef slope ($\beta_f$), reef flat width ($W_reef$), beach slope ($\beta_b$), and seabed roughness ($c_f$). ${L_0}$ is the deep water wave length $L_0=gT_p^2/2pi$, and $T_p$ is the peak period. Beach crest elevation ($z_b$) was fixed at a height of 30 m to focus on run-up as a proxy for coastal inundation.

<img src="resources/nb02_04_profile.png">


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Interpolation
from teslakit.mda import Normalize

from teslakit.plotting.extremes import Plot_ReturnPeriodValidation



## Database and Site parameters

In [ ]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


In [ ]:
# load Hycreww RBF coefficients and sim. variables min. and max.
var_lims, rbf_coeffs = db.Load_HYCREWW()

# reef characteristics
reef_cs = {
    'rslope': 0.0505,
    'bslope': 0.1667,
    'rwidth': 250,
    'cf': 0.0105,
}

In [ ]:
# hycreww interpolation 

def hycreww_runup(xds):
    '''
    Calculates RunUp using hycreww RBFs (level) and linear interpolation (Runup)
    
    var_lims   - hycreww variables min and max limits
    rbf_coeffs - hycreww rbf coefficients
    reef_cs    - reef characteristics
    
    xds       - input dataset (xarray.Dataset with "rbf_vns" variables)
    '''
    
    # parse input to pandas.dataframe
    df = xds[['hs', 'tp', 'dir', 'level', 'hs_lo2']].to_dataframe()
    
    # RBF wave conditions 
    rbf_hs = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
    rbf_hs_lo = [0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05 ]
    rbf_vns = ['level', 'rslope', 'bslope', 'rwidth', 'cf']
    
    # RBF parameters
    ix_sc = [0, 1, 2, 3, 4]
    ix_dr = []
    minis = [var_lims[x][0] for x in rbf_vns]
    maxis = [var_lims[x][1] for x in rbf_vns]

    # add reef characteristics
    for p in reef_cs.keys(): df[p] = reef_cs[p]
        
    # data inside hycrew limits 
    lp = []
    for vn in var_lims.keys():
        ps = (df[vn] > var_lims[vn][0]) & (df[vn] < var_lims[vn][1])
        lp.append(ps)
    ix_in = np.where(np.all(lp, axis=0))[0]  # all variables inside limits
    

    # RBF dataset to interpolate
    ds_in = df.iloc[ix_in]  # index inside limits
    ds_rbf_in = ds_in[rbf_vns]  # rbf variables

    # normalize data
    ds_nm ,_ ,_ = Normalize(ds_rbf_in.values, ix_sc, ix_dr, minis=minis, maxis=maxis)

    # RBF interpolation (with all cases?)
    aux_1 = []
    for rc in rbf_coeffs:
        ro = RBF_Interpolation(rc['constant'], rc['coeff'], rc['nodes'], ds_nm.T)
        aux_1.append(ro)
    ru_z = np.array(aux_1)
    
    
    # TODO upgrade interpolation for
    
    # RU Linear interpolation (hs, hs_lo -> runup)
    aux_2 = []
    for c, (_, r) in enumerate(ds_in.iterrows()):
        vq = griddata((rbf_hs, rbf_hs_lo), ru_z[:,c], (r['hs'], r['hs_lo2']), method='linear')
        aux_2.append(vq)
    ru_in = np.array(aux_2)
    
    # add level to run_up
    ru_in = ru_in + ds_in['level']
    
    # return runup 
    ru_out = np.zeros(len(df.index)) * np.nan
    ru_out[ix_in] = ru_in
    
    xds_ru = xr.Dataset(
        {
            'runup': (('time',), ru_out)
        },
        coords={'time': df.index},
    )

    return xds_ru



## Hycreww RBF Interpolation: Historical

In [ ]:
# Load complete historical data and nearshore waves

data = db.Load_HIST_OFFSHORE(decode_times=True, vns=['level'])  # offshore data (level)
waves = db.Load_HIST_NEARSHORE(decode_times=True, vns=['Hs', 'Tp', 'Dir'])  # nearshore waves

# ensure data share waves time array
data = data.sel(time = waves.time)

# prepare data for hycreww
waves = waves.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
waves['hs_lo2'] = waves['hs']/(1.5613*waves['tp']**2)             # calc. hs_lo2
waves['level'] = data.level                                       # add level

# calculate runup with hycreww
ru_hist = hycreww_runup(waves)

# store historical runup
db.Save_HIST_NEARSHORE(ru_hist)



## Hycreww RBF Interpolation: Simulation

In [ ]:
# Load offshore simulations level 
data = db.Load_SIM_OFFSHORE_all(decode_times=False, vns=['level'])

# iterate  simulations
for n in data.n_sim:
    data_n = data.sel(n_sim=int(n))
    waves_n = db.Load_SIM_NEARSHORE(int(n), vns=['Hs', 'Tp', 'Dir'])
    
    # prepare data for hycreww
    waves_n = waves_n.rename_vars({"Hs": "hs", "Tp": "tp", 'Dir':'dir'})  # rename vars
    waves_n['hs_lo2'] = waves_n['hs']/(1.5613*waves_n['tp']**2)           # calc. hs_lo2
    waves_n['level'] = data_n.level                                       # add level
    
    # calculate runup with hycreww
    ru_sim_n = hycreww_runup(waves_n)
    
    # store simulation runup
    db.Save_SIM_NEARSHORE(ru_sim_n, int(n))
    
    print('simulation {0} processed.'.format(int(n)))




## Methodology Validation: Annual Maxima

In [ ]:
# load all simulations
ru_sims = db.Load_SIM_NEARSHORE_all(vns=['runup'], decode_times=True)


# compare historical and simulations runup annual maxima
hist_A = ru_hist['runup'].groupby('time.year').max(dim='time')
sim_A = ru_sims['runup'].groupby('time.year').max(dim='time')

# Return Period historical vs. simulations
Plot_ReturnPeriodValidation(hist_A, sim_A);
    